# Array programming in NumPy

While many of the array operations that are familiar to you from Python lists will also work with NumPy arrays, an important difference between NumPy arrays and Python lists is how we _should_ operate on them.  Specifically, to use NumPy arrays idiomatically, you sometimes need to think in terms of programming with arrays.  This means that some operations will have results that you don't expect if you're thinking of how Python lists work.

For example, consider the `max` function in Python:

In [ ]:
help(max)

So `max` returns its maximum argument.  If its arguments are lists, it will return the list with the largest first element (if there is a tie, it will return the list from the tie with the largest second element, and so on).  Try it out:

In [ ]:
max([1,1,5,9],[1,2,4,6])

The similarly-named NumPy `maximum` function has a slightly different interface and subtly-different behavior.  While the built-in `max` function will take the maximum from any number of arguments, NumPy's `maximum` takes two.  Furthermore, its result is slightly different, as you can see.

In [ ]:
import numpy as np
np.maximum([1,1,5,9],[1,2,4,6])

(Notice that we invoked `np.maximum` on two Python lists, but it would behave similarly on two NumPy arrays.)

In [ ]:
np.maximum(np.array([1,1,5,9]),np.array([1,2,4,6]))

What did it do?

This may not have been what you expect, but operations that work on entire arrays an element at a time (rather than on entire arrays  as atomic objects) are extremely important in NumPy.  We can see another example in the next two cells:

In [ ]:
[1,2,5,9] * 3

In [ ]:
np.array([1,2,5,9]) * 3

These elementwise operations are useful and they're also far more efficient than code you'd write in Python.  To see why, write a Python function to take the elementwise maximum of two arrays and compare its performance to `np.maximum`.   First, let's set up some test data:

In [ ]:
np.random.seed(0xDA7ABA5E)
ls1 = list(np.random.randint(256, size=5000))
ls2 = list(np.random.randint(256, size=5000))

In [ ]:
%%time

def nmax(la, lb):
    # FIXME: replace this function's body with a function that computes 
    # and returns the elementwise maximum of a and b.  Do not use NumPy.
    return la

for i in range(10000):
    nmax(ls1, ls2)
    pass

If you get stuck on the previous cell, [see the hint](hints/hint02.ipynb).

In [ ]:
%%time

def pmax(a, b):
    return np.maximum(a, b)

for i in range(10000):
    pmax(ls1, ls2)
    pass